<a href="https://colab.research.google.com/github/mourodrigo/CarClassifier/blob/main/CarClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing libs

In [180]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import time

import re #regex

from bs4 import BeautifulSoup as bs # to parse HTML files

print('Libraries imported.')

Libraries imported.


## HTML Reading and parsing

In [138]:
with open('sample_data/cars.html') as file:
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]


### Data parsing

In [139]:
soup = bs(lines[0], 'html') 
html_container = soup.find("div",class_="bq4bzpyk j83agx80 btwxx1t3 lhclo0ds jifvfom9 muag1w35 dlv3wnog enqfppq2 rl04r1d5")
html_cars = soup.findAll("div",class_="b3onmgus ph5uu5jm g5gj957u buofh1pr cbu4d94t rj1gh0hx j83agx80 rq0escxv fnqts5cd fo9g3nie n1dktuyu e5nlhep0 ecm0bbzt")

In [152]:
cars = pd.DataFrame(columns=['year','name', 'value', 'city', 'kms', 'url'])
for item in html_cars:
  name_html = item.find("span",class_="a8c37x1j ni8dbmo4 stjgntxs l9j0dhe7")
  
  year = 0
  name = ""
  if name_html is not None:
    name = name_html.text
    regx_year = re.findall("\d+", name)   
    if len(regx_year) > 0:
      year = regx_year[0]
      year = int(year)
      if year < 2000:
        year = 0

  value_html = item.find("span",class_="d2edcug0 hpfvmrgz qv66sw1b c1et5uql oi732d6d ik7dh3pa ht8s03o8 a8c37x1j keod5gw0 nxhoafnm aigsh9s9 fe6kdd0r mau55g9w c8b282yb d9wwppkn iv3no6db a5q79mjw g1cxx5fr lrazzd5p oo9gr5id")
  value = 0
  if value_html is not None:
    value = re.findall("\d+", value_html.text.replace(".", ""))[0]
    value = int(value)

  city_html = item.find("span",class_="a8c37x1j ni8dbmo4 stjgntxs l9j0dhe7 ltmttdrg g0qnabr5")
  city = ""
  if city_html is not None:
    city = city_html.text
    
  kms_html = item.findAll("span",class_="d2edcug0 hpfvmrgz qv66sw1b c1et5uql oi732d6d ik7dh3pa ht8s03o8 a8c37x1j keod5gw0 nxhoafnm aigsh9s9 fe6kdd0r mau55g9w c8b282yb d9wwppkn mdeji52x e9vueds3 j5wam9gi b1v8xokw m9osqain")
  kms = 99999999999

  if len(kms_html)>1 and kms_html[1] != "  ":
    kms = kms_html[1].text
    regx = re.findall("\d+", kms)   
    if len(regx) > 0:
      kms = regx[0]
      kms = int(kms)
      kms = kms*1000
      kms = int(kms)
      
  for a in item.find_all('a', href=True):
      url = "https://www.facebook.com"+a['href'].split("?")[0]

  if year>0 and kms != '': 
    cars = cars.append({'year':year, 'name':name, 'value':value, 'city':city, 'kms':kms, 'url':url}, ignore_index=True)

In [174]:
cars['yearsOfUse'] = 2022-cars['year']+0.001 

In [181]:
cars['kmsByYear'] = (cars['kms']/cars['yearsOfUse'])
cars.sort_values(by=['kmsByYear'])

,year,name,value,city,kms,url,yearsOfUse,kmsByYear
201,2016,2016 FIAT Strada,69990,"Chapecó, SC",1000,https://www.facebook.com/marketplace/item/1257...,6.001,166.639
15,2019,2019 Volkswagen Polo,89990,"Chapecó, SC",1000,https://www.facebook.com/marketplace/item/6286...,3.001,333.222
7,2017,2017 Chevrolet Onix,70900,"Chapecó, SC",2000,https://www.facebook.com/marketplace/item/9019...,5.001,399.92
157,2015,2015 Honda Fit,59990,"Chapecó, SC",11000,https://www.facebook.com/marketplace/item/5964...,7.001,1571.2
200,2017,2017 Hyundai HB20,55900,"Chapecó, SC",19000,https://www.facebook.com/marketplace/item/3161...,5.001,3799.24
72,2021,2021 Chevrolet Onix,89900,"Chapecó, SC",4000,https://www.facebook.com/marketplace/item/6244...,1.001,3996
199,2019,2019 Hyundai HB20,60900,"Chapecó, SC",12000,https://www.facebook.com/marketplace/item/2901...,3.001,3998.67
208,2015,2015 Harley-Davidson Fat Boy,61000,"Chapecó, SC",30000,https://www.facebook.com/marketplace/item/4294...,7.001,4285.1
197,2020,2020 Toyota Yaris,85000,"Chapecó, SC",10000,https://www.facebook.com/marketplace/item/6489...,2.001,4997.5
155,2019,2019 Renault Captur INTENSE 1.6 FLEX AUTOMÁTIC...,93000,"Xanxerê, SC",16000,https://www.facebook.com/marketplace/item/1593...,3.001,5331.56
